In [1]:
from python_workflow_definition.aiida import write_workflow_json, construct_wg_simple
from python_workflow_definition.jobflow import load_workflow_json
from jobflow.managers.local import run_locally

from aiida import load_profile
load_profile()

workflow_json_filename =  "workflow_simple_aiida.json"

/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
from simple_workflow import add_x_and_y, add_x_and_y_and_z

In [3]:
wg = construct_wg_simple(
    add_x_and_y_func=add_x_and_y, add_x_and_y_and_z_func=add_x_and_y_and_z
)

outputs [{'identifier': 'workgraph.any', 'name': 'result'}]
outputs ['x', 'y', 'z']
outputs [{'identifier': 'workgraph.any', 'name': 'result'}]


In [4]:
write_workflow_json(wg=wg, file_name=workflow_json_filename)


{'nodes': {0: 1,
  1: 2,
  2: 'simple_workflow.add_x_and_y',
  3: 'simple_workflow.add_x_and_y_and_z'},
 'edges': [{'source': 0,
   'target': 2,
   'sourceHandle': None,
   'targetHandle': 'x'},
  {'source': 1, 'target': 2, 'sourceHandle': None, 'targetHandle': 'y'},
  {'source': 2, 'target': 3, 'sourceHandle': 'x', 'targetHandle': 'x'},
  {'source': 2, 'target': 3, 'sourceHandle': 'y', 'targetHandle': 'y'},
  {'source': 2, 'target': 3, 'sourceHandle': 'z', 'targetHandle': 'z'}]}

In [5]:

flow = load_workflow_json(file_name=workflow_json_filename)

In [6]:
result = run_locally(flow)
result

2025-03-15 11:21:45,872 INFO Started executing jobs locally
2025-03-15 11:21:46,028 INFO Starting job - add_x_and_y (31bed0f7-49cd-4e63-9a53-8c9679e74dd2)
2025-03-15 11:21:46,030 INFO Finished job - add_x_and_y (31bed0f7-49cd-4e63-9a53-8c9679e74dd2)
2025-03-15 11:21:46,031 INFO Starting job - add_x_and_y_and_z (7d74164e-3d5b-40d0-b72b-4fc5434a98e1)
2025-03-15 11:21:46,033 INFO Finished job - add_x_and_y_and_z (7d74164e-3d5b-40d0-b72b-4fc5434a98e1)
2025-03-15 11:21:46,033 INFO Finished executing jobs locally


{'31bed0f7-49cd-4e63-9a53-8c9679e74dd2': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))},
 '7d74164e-3d5b-40d0-b72b-4fc5434a98e1': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))}}